In [53]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv
!pip3 install --quiet langchain_experimental langchain_openai


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

## Define our LLM

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
# llm.invoke("Tell me a joke about cats")

AIMessage(content='Why did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d30b0f3-d53e-4f59-8046-776c5bc959df-0', usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Process PDF document

In [9]:
loader = PyPDFLoader('data/Resume-Samples-pages-1.pdf') # Load the resume
pages = loader.load() # Load the pages of the resume
pages

[Document(metadata={'source': 'data/Resume-Samples-pages-1.pdf', 'page': 0, 'page_label': '1'}, page_content=' \n \nDevin C. Debit  \ndevindebit@noemail7.com  973-761-9355 \n  \nCAREER OBJECTIVE \nTo obtain an entry-level position with a regional public accounting firm with a special interest in corporate \ntax accounting and auditing  \n \nEDUCATION \nSeton Hall University, South Orange, NJ  \nMaster of Science in Accounting        May 20xx  \nEligible to sit for the CPA exam with 150 credits \nGPA:  3.83/4.00 \n \nUniversity of South Florida, Tampa, FL   May 20xx \nBachelor of Science in Finance \nGPA: 3.64/4.00  \n \nRELATED COURSEWORK  \nManagerial Accounting and Decision Making    Advanced Corporate Income Tax  \nAdvanced Financial Accounting      Microeconomics Theory and Applications \nCorporate Finance I, II and III      Enterprise Accounting \n \nEXPERIENCE \nAnderson, Smith & Jones, LLC, Roseland, NJ    \nAccounting Intern         May 20xx - September 20xx \n\x87 Assisted wit

## split document

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, # Split the text into chunks
                                               chunk_overlap=200, # Chunk overlap is to ensure that the chunks are not cut off in the middle of a word
                                               length_function=len, # Length function is the function that calculates the length of the text
                                               separators=['\n', '\r\n', '\r', ' ']) # Separators are the characters that are used to separate the text into chunks
chunks = text_splitter.split_documents(pages) # Split the text into chunks

## Create embeddings

In [18]:
def get_embedding_fn(): # Get the embedding function
    embeddings = OpenAIEmbeddings( 
        model="text-embedding-ada-002",
        api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_fn = get_embedding_fn()
test_vector = embedding_fn.embed_query("cat") # Test the embedding function

In [19]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings=embedding_fn)
evaluator.evaluate_strings(prediction="cat", reference="dog") # Evaluate the embedding function

{'score': 0.13704235445734714}

## Create vector database

In [25]:
import uuid

# create a new chroma database from the documents
def create_vector_store(chunks, embedding_fn, vector_store_path):
    # prevent duplicate vector store directories

    #  create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)
    
    # create a new vector store
    vector_store = Chroma.from_documents(documents=unique_chunks,
                                         ids=list(unique_ids),
                                        embedding=embedding_fn,
                                        persist_directory=vector_store_path)
    vector_store.persist()
    return vector_store
                                    

In [26]:
# create a new vector store
vector_score = create_vector_store(chunks=chunks,
                                  embedding_fn=embedding_fn,
                                  vector_store_path="vector_store_chroma")

C:\Users\Owner\AppData\Local\Temp\ipykernel_20320\6781656.py:23: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


## Query for relevant data

In [28]:
# load vector store
vector_store = Chroma(persist_directory="vector_store_chroma",
                      embedding_function=embedding_fn)

In [30]:
# create retriever and get relevant chunks
retriever = vector_store.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the name of the person in this resume?")
relevant_chunks

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(metadata={'page': 0, 'page_label': '1', 'source': 'data/Resume-Samples-pages-1.pdf'}, page_content='manufacturing firm \n \nStephen Roberts, Certified Public Accountant, Raleigh, NC    \nAccoXnWanW¶s AssisWanW        Summer 20xx \n\x87 PUoceVVed clienWV¶ Wa[ UeWXUnV and filed Xp Wo WZo do]en annXal UepoUWV online each monWh \n\x87 Entered payroll data for 10 small businesses using Professional Suite Accounting Write -Up CS \n \nUniversity of South Florida Housing and Residence Life, Miami, FL  \nResident Assistant         August 20xx - May 20xx \n\x87 Managed a university residence hall floor of 55 ethnically diverse undergraduate students \n\x87 Organized and facilitated monthly floor meetings; coordinated weekly programs \n\x87 Worked in Residence Life office, managed building front desk, participated in on call duty  \n \nHONORS AND AWARDS \nToastmasters International of Nashville Gold Medal - Oratory Competition, March 20xx  \nPhi Kappa Phi Honor Society, Member, January 

In [31]:
# prompt template

PROMPT_TEMPLATE = """ 
    You are a recruiter for a company and you are looking for a candidate to fill a position. 
    You have a resume in front of you and you want to know more about the candidate. 
    You want to know the name of the candidate. The resume is attached below. 
    Please provide the name of the candidate.

    If you don't know the answer, say that you don't know. DON'T GUESS OR MAKE UP ANYTHING.

    {context}

    ---

    Answer the question based on the above context: {question}
"""

In [33]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text,
                                question="What is the name of the person in this resume?")
print(prompt)

Human:  
    You are a recruiter for a company and you are looking for a candidate to fill a position. 
    You have a resume in front of you and you want to know more about the candidate. 
    You want to know the name of the candidate. The resume is attached below. 
    Please provide the name of the candidate.

    If you don't know the answer, say that you don't know. DON'T GUESS OR MAKE UP ANYTHING.

    manufacturing firm 
 
Stephen Roberts, Certified Public Accountant, Raleigh, NC    
AccoXnWanW¶s AssisWanW        Summer 20xx 
 PUoceVVed clienWV¶ Wa[ UeWXUnV and filed Xp Wo WZo do]en annXal UepoUWV online each monWh 
 Entered payroll data for 10 small businesses using Professional Suite Accounting Write -Up CS 
 
University of South Florida Housing and Residence Life, Miami, FL  
Resident Assistant         August 20xx - May 20xx 
 Managed a university residence hall floor of 55 ethnically diverse undergraduate students 
 Organized and facilitated monthly floor meetings; coor

## Generate responses

In [35]:
llm.invoke(prompt)

AIMessage(content='The name of the candidate in this resume is Devin C. Debit.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 770, 'total_tokens': 785, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-891c60cd-d110-4dde-b245-d4a1eb042025-0', usage_metadata={'input_tokens': 770, 'output_tokens': 15, 'total_tokens': 785, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## using langchain expression language to generate responses

In [36]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt_template
    | llm
)

## Generate structured responses

In [50]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="The answer to the question.")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question.")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources.")


class ExtractedInfo(BaseModel):
    """Extracted information about the resume:"""
    name: AnswerWithSources  
    email: AnswerWithSources  
    phone: AnswerWithSources  
    education: AnswerWithSources  
    experience:  AnswerWithSources  
    skills: AnswerWithSources  
    summary:  AnswerWithSources  

In [51]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt_template
    | llm.with_structured_output(ExtractedInfo, strict=True)
)

rag_chain.invoke("Give me the name, email, phone number, and education of the person in this resume.")

c:\Users\Owner\OneDrive\Desktop\Resume Parser\resume-parser\env\Lib\site-packages\langchain_openai\chat_models\base.py:1363: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


ExtractedInfo(name=AnswerWithSources(answer='Devin C. Debit', sources='Devin C. Debit', reasoning='The name of the candidate is clearly stated at the top of the resume.'), email=AnswerWithSources(answer='devindebit@noemail7.com', sources='devindebit@noemail7.com', reasoning="The email is provided directly beneath the candidate's name on the resume."), phone=AnswerWithSources(answer='973-761-9355', sources='973-761-9355', reasoning='The phone number is listed alongside the email address in the resume.'), education=AnswerWithSources(answer='Seton Hall University, Master of Science in Accounting; University of South Florida, Bachelor of Science in Finance', sources='Seton Hall University, South Orange, NJ Master of Science in Accounting; University of South Florida, Tampa, FL Bachelor of Science in Finance', reasoning='The education section of the resume lists degrees from these universities.'), experience=AnswerWithSources(answer='The candidate has experience as an Accounting Intern at A

## Transform response into a dataframe

In [52]:
structured_res = rag_chain.invoke("Give me the name, email, phone number, and education of the person in this resume.")
df = pd.DataFrame([structured_res.dict()])

# transform into a table with three rows: answer, source, and reasoning
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# create new dataframe with three rows: answer, source, and reasoning
structured_res_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['Answer', 'Source', 'Reasoning'])

structured_res_df

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


,name,email,phone,education,experience,skills,summary
Answer,Devin C. Debit,devindebit@noemail7.com,973-761-9355,"Seton Hall University, Master of Science in Ac...","Accounting Intern at Anderson, Smith & Jones, ...","Tax accounting, auditing, bookkeeping, and tea...",Entry-level candidate focused on corporate tax...
Source,Devin C. Debit,devindebit@noemail7.com,973-761-9355,"Seton Hall University, South Orange, NJ Master...","Anderson, Smith & Jones, LLC, Roseland, NJ; St...",Related coursework indicates accounting skills...,Career objective states interest in corporate ...
Reasoning,The name is explicitly mentioned at the top of...,The email address is provided directly in the ...,The phone number is listed in the resume.,The education section of the resume outlines t...,The experience section lists the candidate's r...,Inferred from experience and stated career obj...,The career objective provides a clear summary ...
